In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import numpy as np
import cv2


In [ ]:

# Load data
def load_data(path):
    images = []
    labels = []
    for label_dir in os.listdir(path):
        full_dir = os.path.join(path, label_dir)
        if not os.path.isdir(full_dir):
            continue
        for file in os.listdir(full_dir):
            img = cv2.imread(os.path.join(full_dir, file))
            if img is not None:
                img = cv2.resize(img, (224, 224)) / 255.0
                images.append(img)
                labels.append(int(label_dir))  # label_dir should be numeric or mapped
    return np.array(images), np.array(labels)

train_dir = "Bananadetetion/train"
val_dir = "Bananadetetion//valid"

x_train, y_train = load_data(train_dir)
x_val, y_val = load_data(val_dir)

# Simple CNN model
model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(set(y_train)), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("banana_model.tflite", "wb") as f:
    f.write(tflite_model)
